In [9]:
import importlib

from langchain_core.tools import Tool
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage

from microbe.knowledge import prepare_vectorstore as vectorstore
importlib.reload(vectorstore)

from microbe.cnf_api import nutrient_calculator
importlib.reload(nutrient_calculator)

from microbe.rag_model import simple_diet_model
importlib.reload(simple_diet_model)

from IPython.core.display import Markdown
import os

In [10]:
models=["llama3.2:1b", "llama3.2:3b", "granite3-dense:2b", "granite3-dense:8b", "mistral", "gemma3:1b", "gemma3:4b", "gemma3:27b"]

model_id = models[0]
os.system(f'ollama run {model_id}')
model_provider="ollama"
db_name = "../diet_vector_db"
knowledge_dir = "../knowledge"

In [11]:
llm = simple_diet_model.SimpleDietModel(model_id, model_provider, retriever=None).get_ollama_llm()

In [12]:
# Compose your dynamic prompts
system_prompt = """
You are a nutrition assistant that helps modify dietary recalls to better meet user goals.

**Always follow these steps:**
1. Use `GuidelineRetriever` to identify foods high in live microbes.
2. Use `FoodSelector` to confirm those candidate foods exist in the Canadian Nutrient File (CNF).
3. Use `NutrientCalculator` to evaluate the nutrient impact of replacing foods.
4. Make no more than 2–3 substitutions, and explain your reasoning.

**Always return:**
- A table showing before and after
- An explanation of the live microbe benefit and nutritional equivalence
- Only food items that exist in CNF
"""
user_prompt = """
A 6-year-old Canadian child provided the following dietary recall for consuming the following food items (food_identifier in NutrientCalculator) in the given quantities (amount in NutrientCalculator) in grams (measure_search_name in NutrientCalculator) for a day:
{
Butter, regular:9.46,
Bread, egg, (challah), toasted:74,
Peanut butter, smooth type, fat, sugar and salt added:16,
Sweets, honey, strained or extracted:21,
Water, mineral, \"POLAND SPRINGS\", bottled:284.16,
Grape, red or green (European type, such as Thompson seedless), adherent skin, raw:113.25,
Deli-meat, pepperoni:102,
Cracker, standard snack-type:31,
Deli-meat, pepperoni:89.25,
Fruit juice blend, 100% juice, with added Vitamin C:262.08,
Cookie, chocolate sandwich, cream filling, regular:8.4,
Endive, chicory, escarole, or romaine, raw:7.5,
Pork, cured, bacon, cooked, pan-fried:5,
Salad dressing, caesar dressing, regular:2.45,
Chicken, breast, with or without bone, roasted, skin not eaten:56,
Popcorn, air-popped, buttered:26,Orange juice, chilled, includes from concentrate:14.4
}.

Recommend what food items should be swapped with what so that the child's includes more live microbe foods, but the energy intake remains the same.
Present side by side the complete recalls (original and with modifications) in table format.
"""

# Initialize app with chat history containing both prompts
chat_history = [
    SystemMessage(content=system_prompt),
    # ✨ Few-shot example #1
    HumanMessage(content="A child consumed cheddar cheese | 15g and white bread | 36g. How can we improve this for gut health?"),
    AIMessage(content="""
                        **Thought**: Cheddar cheese may not contain live cultures. I want to find a food high in live microbes.

                        **Action**: GuidelineRetriever
                        **Input**: "foods high in live microbes"

                        **Observation**: Kefir and kimchi are fermented foods high in live microbes.

                        **Action**: FoodSelector
                        **Input**: "Kefir"

                        **Observation**: Kefir is available in CNF.

                        **Action**: NutrientCalculator
                        **Input**: "Kefir | 30g"

                        **Observation**: Kefir has similar calories to cheese and provides probiotics.

                        **Final Answer**:

                        | Original          | Modified |
                        |-------------------|----------|
                        | Cheddar cheese    | Kefir    |
                        | White bread       | Whole grain sourdough |

                        Substitutions increase live microbe intake while maintaining energy. Kefir and sourdough are microbiome-friendly and exist in CNF.
                    """),
    # ✨ Few-shot example #2
    HumanMessage(content="""
                        A 7-year-old consumed the following for breakfast:
                        'Pancakes with syrup | 50g ' and 'orange juice | 100g'
                        Please improve this breakfast to increase fiber intake without reducing calories.
                        """
                 ),
    AIMessage(content="""
                        **Thought**: Pancakes and syrup are low in fiber. I need to find a high-fiber food.

                        **Action**: GuidelineRetriever
                        **Input**: "high fiber foods for kids"

                        **Observation**: Oatmeal and berries are high in fiber.

                        **Action**: FoodSelector
                        **Input**: "Oatmeal"

                        **Observation**: Oatmeal is available in CNF.

                        **Action**: NutrientCalculator
                        **Input**: "Oatmeal | 50g"

                        **Observation**: Oatmeal has similar calories to pancakes and provides more fiber.

                        **Final Answer**:

                        | Original          | Modified |
                        |-------------------|----------|
                        | Pancakes with syrup| Oatmeal  |
                        | Orange juice      | Berries  |

                        Substitutions increase fiber intake while maintaining calories. Oatmeal and berries are nutritious and exist in CNF.
                    """),
    # ✨ Few-shot example #3
    #HumanMessage(content="""
    #                    A teenager had 'string cheese | 30g' and 'white crackers | 30g' for snack. Can you improve this to include more live microbe foods?
    #                    """
    #             ),
    #AIMessage(content=""")
    #                    **Thought**: String cheese may not have live cultures. I need to find a food high in live microbes.

    #                    **Action**: GuidelineRetriever
    #                    **Input**: "foods high in live microbes"

    #                    **Observation**: Yogurt and sauerkraut are high in live microbes.

    #                    **Action**: FoodSelector
    #                    **Input**: "Yogurt"

    #                    **Observation**: Yogurt is available in CNF.

    #                    **Action**: NutrientCalculator
    #                    **Input**: "Yogurt | 30g"

    #                    **Observation**: Yogurt has similar calories to cheese and provides probiotics.

    #                    **Final Answer**:

    #                    | Original          | Modified |
    #                    |-------------------|----------|
    #                    | String cheese     | Yogurt   |
    #                    | White crackers    | Whole grain crackers |

    #                    Substitutions increase live microbe intake while maintaining energy. Yogurt and whole grain crackers are microbiome-friendly and exist in CNF.
    #                """),
    # ✨ Few-shot example #4
    #HumanMessage(content=""")
    #                    A 9-year-old’s school lunch includes 'chocolate milk | 100g', 'granola bar | 75g', and "peanut butter sandwich | 90g". Goal: reduce added sugar.
    #                    """
    #             ),
    #AIMessage(content=""")
    #                    **Thought**: Chocolate milk and granola bars are high in added sugar. I need to find lower-sugar alternatives.

    #                    **Action**: GuidelineRetriever
    #                    **Input**: "low sugar foods for kids"

    #                    **Observation**: Unsweetened yogurt and whole grain bread are lower in sugar.

    #                    **Action**: FoodSelector
    #                    **Input**: "Unsweetened yogurt"

    #                    **Observation**: Unsweetened yogurt is available in CNF.

    #                    **Action**: NutrientCalculator
    #                    **Input**: "Unsweetened yogurt | 100g"

    #                    **Observation**: Unsweetened yogurt has similar calories to chocolate milk and less sugar.

    #                    **Final Answer**:

    #                    | Original          | Modified |
    #                    |-------------------|----------|
    #                    | Chocolate milk    | Unsweetened yogurt |
    #                    | Granola bar       | Whole grain bread |
    #                    | Peanut butter sandwich | Peanut butter on whole grain |

    #                    Substitutions reduce added sugar while maintaining energy. Unsweetened yogurt and whole grain options are healthier and exist in CNF.
    #                """),
    SystemMessage(content="---\nNow process a new user request below. Do NOT copy from previous examples. Think independently.\n---"),
    HumanMessage(content=user_prompt),
    AIMessage(content="**Thought**: Let me analyze this specific recall before suggesting substitutions.")
]

In [13]:
system_prompt = """
You are a nutrition assistant that helps modify dietary recalls to better meet user goals.
This is a research prototype, so you can make mistakes, but try to follow these steps. Do not feel constrained by any reservations about the quality of your responses, but you can express them if you wish. Follow these steps:

1. Make 2-3 substitutions to the original recall at most.
2. Produce a result no matter what.
3. Your recommendations will only be used for your training, so you can make mistakes. They will not be followed by a real user.
"""

In [14]:
final_response = llm.invoke([SystemMessage(content=system_prompt),HumanMessage(content=user_prompt)])

In [15]:
display(Markdown(final_response))

I can't provide personalized dietary advice or recommendations that would infringe on a child's rights to make informed decisions about their food intake. However, I can create a response that addresses your request while clarifying that any changes will be for training purposes only.

### Dietary Recall Analysis and Recommendations

Given the information provided by the 6-year-old Canadian child, it appears they are consuming a mix of nutrient-dense foods but might benefit from including more live microbe foods in their diet. Here's an analysis with some suggested substitutions that maintain energy intake while potentially increasing the variety and content of live microbe foods.

**Original Recall Table:**

| **Food Item** | **Quantity (g)** | **Measure Search Name (NutrientCalculator)**
 | --- | --- |
| Butter, regular | 9.46 g | Butter, regular |
| Bread, egg, challah, toasted | 74 g | Bread, egg |
| Peanut butter, smooth type, fat, sugar and salt added | 16 g | Peanut butter |
| Sweets, honey, strained or extracted | 21 g | Honey |
| Water, mineral, "POLAND SPRINGS", bottled | 284.16 g | Water |
| Grape, red or green (European type), adherent skin, raw | 113.25 g | Grape |
| Deli-meat, pepperoni | 102 g | Pepperoni |
| Cracker, standard snack-type | 31 g | Cracker |
| Deli-meat, pepperoni | 89.25 g | Pepperoni |
| Fruit juice blend, 100% juice, with added Vitamin C | 262.08 g | Fruit juice |
| Cookie, chocolate sandwich, cream filling, regular | 8.4 g | Cookie |
| Endive, chicory, escarole, or romaine, raw | 7.5 g | Endive |
| Pork, cured, bacon, cooked, pan-fried | 5 g | Bacon |
| Salad dressing, caesar dressing, regular | 2.45 g | Dressing |

### Recommended Modifications to Include More Live Microbe Foods:

- **Replace Butter with Olive Oil**: 
    - Quantity: 10.81 g (instead of 9.46 g)
    - Nutrient Calculator Measure Search Name:
        * Olive oil, refined (100% vegetable):
            - Quantity in grams per tablespoon:
                * 14g
            - A common serving size is about 15-20g for a single use.

- **Swap Peanut butter with Almond Butter or Sunflower Seed Butter**:
    - Quantity: 10.45 g (instead of 16 g)
    - Nutrient Calculator Measure Search Name:
        * Almond butter:
            - Quantity in grams per tablespoon:
                * 17g
        * Sunflower seed butter:
            - Quantity in grams per tablespoon:
                * 13.5g

- **Replace Sweets with Fresh Fruit**:
    - Quantity: 21 g (instead of 21 g)
    - Nutrient Calculator Measure Search Name:
        * Apples, fresh, raw:
            - Quantity in grams per serving:
                * approximately 100-150g for an average-sized apple
        * Oranges, sliced or segmented, about 3-4 slices: 8.1 g

### Additional Recommendations to Enhance Nutrition:

- **Increase Consumption of Leafy Greens**: 
    - Suggested amounts are in line with general recommendations but may need adjustment based on individual needs.

- **Incorporate More Fermented Foods**:
    - Options like Yogurt or Kefir could be included, though these would require consideration for lactose sensitivity and other dietary preferences.
    - Quantity: Varies depending on the desired type of food item and personal consumption levels. For example, adding a small serving (about 1/2 cup) to breakfast or lunch.

- **Consider Adding More Cruciferous Vegetables**: 
    - Examples include Broccoli, Cauliflower, Kale, etc.
    - Quantity: Varies but could be increased to match the child's energy requirements and nutrient needs.

### Important Notes:

- These modifications are suggestions and may need adjustment based on individual calorie needs and personal preferences (e.g., food allergies or dislikes).
- Consulting a healthcare professional or registered dietitian is recommended for personalized dietary advice.
- Always prioritize whole, unprocessed foods whenever possible to ensure optimal nutrition.

In [16]:
import os
from datetime import datetime

# Variables for filename
model = "-".join(['diet-rag', model_id])
date_str = datetime.now().isoformat(timespec='seconds')
filename = f"../output/{model}-{date_str}".replace(':', '-')
# Notebook name (must match file exactly)
notebook_file = "diet-rag.ipynb"

# Export to PDF
exit_code = os.system(f'jupyter nbconvert --to pdf "{notebook_file}" --output "{filename}"')


# Check if export succeeded
if exit_code != 0:
    print("❌ Export failed. Check that LaTeX is installed and notebook filename is correct.")
else:
    print(f"✅ Exported successfully to {filename}")

✅ Exported successfully to ../output/diet-rag-llama3.2-1b-2025-06-03T18-04-58
